# Data Cleaning

In [ ]:
import pandas as pd

df = pd.read_excel("Outlet X final.xlsx")


In [ ]:
df['Calendar Year Week'] = df['Calendar Year Week'].astype(str)
df['Fiscal Year Week'] = df['Fiscal Year Week'].astype(str)
df[['Fiscal Week', 'Fiscal Year']] = df['Fiscal Year Week'].str.split('.', expand=True)
df['Fiscal Week'] = df['Fiscal Week'].astype(int)
df['Fiscal Year'] = df['Fiscal Year'].astype(int)

In [ ]:
# Fill NaN values with zeros in the numerical columns
numerical_columns = [
    'Delta In Transit Qty', 
    'Sales Qty',  
]

for column in numerical_columns:
    df[column] = df[column].fillna(0)

# Check if there are any NaN values left in the dataframe
nan_values = df.isna().sum()

nan_values

In [ ]:
# Convert 'Delta In Transit Qty', 'Sales Qty', and 'Sales Qty (incl returns)' columns to integers
df['Delta In Transit Qty'] = df['Delta In Transit Qty'].astype(int)
df['Delta In Transit Qty'] = df['Delta In Transit Qty'].apply(lambda x: 0 if x < 0 else x)
df['Sales Qty'] = df['Sales Qty'].astype(int)
# Convert Calendar Year Week and Calendar Day into datetime format
df['Calendar Day'] = pd.to_datetime(df['Calendar Day'])
#make a month column from calendar day column 
df['Month'] = df['Calendar Day'].dt.month
df['Month Name'] = df['Calendar Day'].dt.strftime('%B')

# Make Sales Qty and Sales Qty (incl returns) positive
df['Sales Qty'] = df['Sales Qty'].abs()

In [ ]:
# check if opening and colosing inventory is <0 
negative_opening_inventory = df[df['Opening Balance Qty'] < 0].shape[0]
negative_closing_inventory = df[df['Closing Balance Qty'] < 0].shape[0]

negative_opening_inventory, negative_closing_inventory

In [ ]:
# check when the opening balance qty is negative but sales qty is positive
negative_opening_positive_sales = df[(df['Opening Balance Qty'] == 0) & (df['Sales Qty'] > 0)].shape[0] 
negative_opening_positive_sales

In [ ]:
#Inv_yn column if the opening balance qty is greater than 0 then 1 else 0
df['Inv_yn'] = df['Opening Balance Qty'].apply(lambda x: 1 if x > 0 else 0)

## Data Quality Check

In [ ]:
#code redundant can be deleted
df['Inv_yn'] = df['Opening Balance Qty'].apply(lambda x: 1 if x > 0 else 0) #Condition: Opening inventory =0 but sales quantity is not equal to 0
mask = df['Inv_yn'] == 0

# Check if any 'Sales Qty' values are non-zero when 'Inv_yn' is 0
condition_check = (df.loc[mask, 'Sales Qty'] > 0).any()

# If condition_check is False, it means there are some rows where 'Inv_yn' is 0 but 'Sales Qty' is not 0
# If condition_check is True, it means that whenever 'Inv_yn' is 0, 'Sales Qty' is also 0
print(not condition_check)

# Create a DataFrame where 'Inv_yn' is 0 but 'Sales Qty' is not 0
df_inconsistent = df[mask & (df['Sales Qty'] > 0)]

# Count the number of inconsistent rows
num_inconsistent = len(df_inconsistent)

print(num_inconsistent)

In [ ]:
# Create a boolean mask where 'Inv_yn' is 0 and 'Closing Balance Qty' is greater than 0 and 'Delta In Transit Qty' is greater than 0
mask = (df['Inv_yn'] == 0) & (df['Closing Balance Qty'] > 0) & (df['Delta In Transit Qty'] > 0) & (df['Sales Qty'] > 0)
# Update the 'Opening Balance Qty' column to 'Closing Balance Qty' + 'Sales Qty' where the mask is True
df.loc[mask, 'Opening Balance Qty'] = df['Closing Balance Qty'] + df['Sales Qty']

In [ ]:
#Inv_yn column if the opening balance qty is greater than 0 then 1 else 0
df['Inv_yn'] = df['Opening Balance Qty'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
mask = df['Inv_yn'] == 0

# Check if any 'Sales Qty' values are non-zero when 'Inv_yn' is 0
condition_check = (df.loc[mask, 'Sales Qty'] != 0).any()

# If condition_check is False, it means there are some rows where 'Inv_yn' is 0 but 'Sales Qty' is not 0
# If condition_check is True, it means that whenever 'Inv_yn' is 0, 'Sales Qty' is also 0
print(not condition_check)

# Create a DataFrame where 'Inv_yn' is 0 but 'Sales Qty' is not 0
df_inconsistent = df[mask & (df['Sales Qty'] != 0)]

# Count the number of inconsistent rows
num_inconsistent = len(df_inconsistent)

print(num_inconsistent)

In [ ]:
# Create a new column 'Stockout' that is True when both 'Inv_yn' and 'Sales Qty' are 0
df['Stockout'] = (df['Inv_yn'] == 0) & (df['Sales Qty'] == 0)

In [ ]:
df_2022 = df[df['Fiscal Year'] == 2022]
df_2023 = df[df['Fiscal Year'] == 2023]

#calculate the percentage of stockouts in the filtered dataframe for 2022 and 2023
stockout_percentage_2022 = (df_2022['Stockout'].sum() / len(df_2022)) * 100
stockout_percentage_2023 = (df_2023['Stockout'].sum() / len(df_2023)) * 100

print(stockout_percentage_2022, stockout_percentage_2023)

In [ ]:
df = df.reindex(columns=['Calendar Day','Weekday', 'Fiscal Year', 'Fiscal Week', 'Fiscal Year Week', 'Calendar Year Week', 'Month', 'Month Name', 'FMS Site', 'FMS Site Name', 'Gender','Article: MH5 Product Group', 'Article: MH6 Sub Product Group', 'Article: Generic ', 'Article: Generic Name', 'Article: Color Description', 'Article', 'Article Name', 'Article: Size ', 'Opening Balance Qty', 'Delta In Transit Qty', 'Sales Qty', 'Closing Balance Qty', 'Inv_yn', 'Stockout'])
df.head()

## Identifying the stockouts correctly

In [ ]:
# Create a pivot table to count 'Stockout' days
stockout_days_2022 = df_2022.pivot_table(index=['Article', 'Article Name'], 
                                         columns='Stockout', 
                                         aggfunc='size', 
                                         fill_value=0)

stockout_days_2023 = df_2023.pivot_table(index=['Article', 'Article Name'], 
                                         columns='Stockout', 
                                         aggfunc='size', 
                                         fill_value=0)

# Calculate the total number of days each article was stockout in 2022 and 2023
stockout_days_2022['Total Days Stockout'] = stockout_days_2022[True]
stockout_days_2023['Total Days Stockout'] = stockout_days_2023[True]

# Drop the False column as it's not needed
stockout_days_2022.drop(columns=False, inplace=True)
stockout_days_2023.drop(columns=False, inplace=True)

# Reset the index so 'Article' and 'Article Name' become columns again
stockout_days_2022.reset_index(inplace=True)
stockout_days_2023.reset_index(inplace=True)

#drop the true column from the stockout_days_2022 and stockout_days_2023
stockout_days_2022.drop(columns=True, inplace=True)
stockout_days_2023.drop(columns=True, inplace=True)

stockout_days_2022.head(), stockout_days_2023.head()   

In [ ]:
# Find the article with 371 total days of stockout in 2023 
article_371_days_2023 = stockout_days_2023[stockout_days_2023['Total Days Stockout'] == 371]

# Display the article number
print(article_371_days_2023['Article'].values)

# Count the number of articles with 371 days of stockout in 2023
num_articles_371_days_2023 = len(article_371_days_2023)

print(num_articles_371_days_2023)

In [ ]:
# Find the article with 364 total days of stockout in 2022
article_364_days_2022 = stockout_days_2022[stockout_days_2022['Total Days Stockout'] == 364]

# Display the article number
print(article_364_days_2022['Article'].values)

# Count the number of articles with 364 days of stockout in 2022
num_articles_364_days_2022 = len(article_364_days_2022)

print(num_articles_364_days_2022)

### Dropped Articles with complete stockout in 2022 and 2023

In [ ]:
# drop the rows with articles with 364 days of stockout in 2022 from the stockout_days_2022 and from dataframe df for only fiscal year 2022 since it contains data with both year 2022 and 2023.
mask = df['Article'].isin(article_364_days_2022['Article']) & (df['Fiscal Year'] == '2022')
df.drop(df[mask].index, inplace=True)

# Drop the rows with articles with 364 days of stockout in 2022 from the stockout_days_2022 DataFrame
mask = stockout_days_2022['Article'].isin(article_364_days_2022['Article'])
stockout_days_2022.drop(stockout_days_2022[mask].index, inplace=True)
stockout_days_2022.reset_index(drop=True, inplace=True)

# drop the rows with articles with 371 days of stockout in 2023 from the stockout_days_2023 and from dataframe df for only fiscal year 2023 since it contains data with both year 2022 and 2023.
mask = df['Article'].isin(article_371_days_2023['Article']) & (df['Fiscal Year'] == '2023')
df.drop(df[mask].index, inplace=True)   

# Drop the rows with articles with 371 days of stockout in 2023 from the stockout_days_2023 DataFrame
mask = stockout_days_2023['Article'].isin(article_371_days_2023['Article'])
stockout_days_2023.drop(stockout_days_2023[mask].index, inplace=True)

# Reset the index of the stockout_days_2023 DataFrame
stockout_days_2023.reset_index(drop=True, inplace=True)


In [ ]:
# number of unique articles in df for fiscal year 2022 and 2023
unique_articles_2022 = df[df['Fiscal Year'] == 2022]['Article'].nunique()
unique_articles_2023 = df[df['Fiscal Year'] == 2023]['Article'].nunique()

unique_articles_2022, unique_articles_2023

In [ ]:
#histogram of the total days stockout in 2022 and 2023
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(stockout_days_2022['Total Days Stockout'], bins=20, color='skyblue')
plt.title('Total Days Stockout in 2022')
plt.xlabel('Number of Days')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(stockout_days_2023['Total Days Stockout'], bins=20, color='salmon')
plt.title('Total Days Stockout in 2023')
plt.xlabel('Number of Days')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

### Phase in fix

In [ ]:
df['Calendar Day'] = pd.to_datetime(df['Calendar Day'])
# Sort the DataFrame by 'Article' and 'Calendar Day'
df = df.sort_values(by=['Article', 'Calendar Day'])
first_nonzero_inventory_day = df[df['Opening Balance Qty'] > 0].groupby('Article')['Calendar Day'].min()

# Define the start and end of February 2022
february_2022_start = pd.Timestamp('2022-02-01')
february_2022_end = pd.Timestamp('2022-02-28')

# Adjust the first_nonzero_inventory_day to consider any day in February as potentially already in stock
first_nonzero_inventory_day_adjusted = first_nonzero_inventory_day.apply(
    lambda x: pd.Timestamp('2022-01-31') if february_2022_start <= x <= february_2022_end else x
)



In [ ]:
# Define a function to check for stockouts
def check_stockout(row):
    if row['Article'] in first_nonzero_inventory_day_adjusted:
        # Only flag as stockout if the date is after the first adjusted arrival and both 'Inv_yn' and 'Sales Qty' are 0
        return row['Calendar Day'] >= first_nonzero_inventory_day_adjusted[row['Article']] and row['Inv_yn'] == 0 and row['Sales Qty'] == 0
    else:
        # If the article has never arrived (according to adjusted dates), it cannot be a stockout
        return False
    
 # Apply the function across the DataFrame
df['Stockout1'] = df.apply(check_stockout, axis=1)

In [ ]:
#see the rows where the opening balance qty is 0 while the stockout1 is false
mask = (df['Opening Balance Qty'] == 0) & (df['Stockout1'] == False)

# View the rows that will be dropped
rows_to_drop = df[mask]
print(rows_to_drop)

In [ ]:
# Group by 'Article' and 'Article Name' columns and count the number of days each article was stockout
stockout_days_article = df.groupby(['Article', 'Article Name', 'Stockout1']).size().unstack(fill_value=0)

# Calculate the total number of days each article was stockout
stockout_days_article['Total Days Stockout'] = stockout_days_article[True]

# Drop the 'False' and 'True' columns
stockout_days_article.drop(columns=[False, True], inplace=True)

# Sort the DataFrame by 'Total Days Stockout' in descending order
stockout_days_article = stockout_days_article.sort_values(by='Total Days Stockout', ascending=False)

stockout_days_article.head(20)

## Phase out fix

In [ ]:
# removing these articles and not updating the stockout since we dont want to calculate the lost sales of the articles that are phased out or not replenished

df = df.sort_values(['Article', 'Calendar Day'])
# Group by 'Article' and calculate the cumulative sum of consecutive stockout days
df['Cumulative_Stockout'] = df.groupby('Article')['Stockout1'].cumsum() - df.groupby('Article')['Stockout1'].cumsum().where(~df['Stockout1']).ffill().fillna(0)

# Find the maximum number of consecutive stockout days for each article
max_consecutive_stockout = df.groupby('Article')['Cumulative_Stockout'].max().reset_index()

# Filter articles with a maximum of at least 90 consecutive stockout days
articles_with_long_stockouts = max_consecutive_stockout[max_consecutive_stockout['Cumulative_Stockout'] >= 30]

# This gives you a DataFrame with articles that have had at least one 30-day consecutive stockout period
articles_with_long_stockouts.head()

In [ ]:
# Calculate cumulative stockouts and reset the counter after a replenishment
df['Cumulative_Stockout'] = (df.groupby(['Article', 'Article Name'])['Stockout1']
                              .cumsum() - df.groupby(['Article', 'Article Name'])['Stockout1']
                              .cumsum().where(~df['Stockout1']).ffill().fillna(0))

# Identify all the unique articles that have ever had a stockout
articles_stocked_out = df[df['Stockout1']]['Article'].unique()

# Initialize an empty list to hold articles that meet the criteria
articles_no_replenishment_after_stockout = []

# Loop through each article that had a stockout
for article in articles_stocked_out:
    # Filter the DataFrame for the current article
    article_df = df[df['Article'] == article]
    # Check if there's any period of 90 consecutive days of stockout
    if any(article_df['Cumulative_Stockout'] >= 30):
        # Find the last stockout day
        last_stockout_day = article_df[article_df['Cumulative_Stockout'] >= 30]['Calendar Day'].max()
        # Check if there's any replenishment after the last stockout day
        replenishment_after_stockout = article_df[(article_df['Calendar Day'] > last_stockout_day) & 
                                                  (article_df['Opening Balance Qty'] > 0)]
        # If there's no replenishment, add the article to the list
        if replenishment_after_stockout.empty:
            articles_no_replenishment_after_stockout.append(article)

# Create a DataFrame with the articles and their names that had no replenishment after a 90-day stockout
final_df = df[df['Article'].isin(articles_no_replenishment_after_stockout)][['Article', 'Article Name']].drop_duplicates()


### Phase out dropped articles check 

In [ ]:
#drop the article numbers in final_df from the original dataframe df
df = df[~df['Article'].isin(final_df['Article'])]

#reset the index of the final_df
final_df.reset_index(drop=True, inplace=True)

In [ ]:
unique_articles_2022 = df[df['Fiscal Year'] == 2022]['Article'].nunique()
unique_articles_2023 = df[df['Fiscal Year'] == 2023]['Article'].nunique()

unique_articles_2022, unique_articles_2023

In [ ]:
#calculate the percentage of stockouts in the filtered dataframe
stockout_percentage = (df['Stockout'].sum() / len(df)) * 100

print(stockout_percentage)

#seperately for year fiscal 2022 and 2023 for the filtered dataframe
df_2022 = df[df['Fiscal Year'] == 2022]
df_2023 = df[df['Fiscal Year'] == 2023]

#calculate the percentage of stockouts in the filtered dataframe for 2022 and 2023
stockout_percentage_2022 = (df_2022['Stockout'].sum() / len(df_2022)) * 100
stockout_percentage_2023 = (df_2023['Stockout'].sum() / len(df_2023)) * 100

print(stockout_percentage_2022, stockout_percentage_2023)

In [ ]:
#Stockout days per aticle after the phase in fix
# Group by 'Article' and 'Article Name' columns and count the number of days each article was stockout
stockout_days_article = df.groupby(['Article', 'Article Name', 'Stockout1']).size().unstack(fill_value=0)

# Calculate the total number of days each article was stockout
stockout_days_article['Total Days Stockout'] = stockout_days_article[True]

# Drop the 'False' and 'True' columns
stockout_days_article.drop(columns=[False, True], inplace=True)

# Sort the DataFrame by 'Total Days Stockout' in descending order
stockout_days_article = stockout_days_article.sort_values(by='Total Days Stockout', ascending=False)
stockout_days_article.head(20)

In [ ]:
# Group by 'Article' and 'Fiscal Year Week' and calculate the number of days each article was stockout in a week 
stockout_days_week = df.groupby(['Article', 'Fiscal Year Week'])['Stockout1'].sum().reset_index()
stockout_days_week.head()
# Merge the original DataFrame with the stockout_days_week DataFrame
df = pd.merge(df, stockout_days_week, how='left', on=['Article', 'Fiscal Year Week'])

# The new column 'Stockout1_y' in df now represents the total stockout days in a week for each day
df.rename(columns={'Stockout1_y': 'Stockout Days in Week', 'Stockout1_x': 'Stockout1'}, inplace=True)

# Visualizations stockout, sales, inventory

## Stockout pattern

In [ ]:
#How can i calculate the weekly stockout percent for fiscal year 2022 and 2023 in subplots nexts to each other by fiscal week

# Filter data for 2022 and 2023
df_2022 = df[df['Fiscal Year'] == 2022]
df_2023 = df[df['Fiscal Year'] == 2023]

# Calculate weekly stockout percent
stockout_2022_weekly = df_2022.groupby('Fiscal Week')['Stockout1'].mean() * 100
stockout_2023_weekly = df_2023.groupby('Fiscal Week')['Stockout1'].mean() * 100

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot data for 2022
axs[0].bar(stockout_2022_weekly.index, stockout_2022_weekly.values)
axs[0].set_title('2022 Weekly Stockout Percent')
axs[0].set_xlabel('Fiscal Week')
axs[0].set_ylabel('Stockout Percent')
axs[0].set_xticks(range(1, 52,2))

# Plot data for 2023
axs[1].bar(stockout_2023_weekly.index, stockout_2023_weekly.values)
axs[1].set_title('2023 Weekly Stockout Percent')
axs[1].set_xlabel('Fiscal Week')
axs[1].set_ylabel('Stockout Percent')
axs[1].set_xticks(range(1, 53,2))


plt.tight_layout()
plt.show()

In [ ]:
# Filter data for 2022 and 2023
df_2022 = df[df['Fiscal Year'] == 2022]
df_2023 = df[df['Fiscal Year'] == 2023]

# Calculate weekly sales sum
sales_2022_weekly = df_2022.groupby('Fiscal Week')['Sales Qty'].sum()
sales_2023_weekly = df_2023.groupby('Fiscal Week')['Sales Qty'].sum()

sales_2022_weekly = sales_2022_weekly / sales_2022_weekly.sum()
sales_2023_weekly = sales_2023_weekly / sales_2023_weekly.sum()

# Plot the sales proportion for 2022 and 2023 in subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot sales proportions for 2022
axs[0].bar(sales_2022_weekly.index, label='2022')
axs[0].set_title('Sales Proportions for 2022')
axs[0].set_xlabel('Fiscal Week')
axs[0].set_ylabel('Sales Proportions')
axs[0].legend()

# Plot sales proportions for 2023
axs[1].bar(sales_2023_weekly.index, label='2023')
axs[1].set_title('Sales Proportions for 2023')
axs[1].set_xlabel('Fiscal Week')
axs[1].set_ylabel('Sales Proportions')
axs[1].legend()

# Show the plot
plt.tight_layout()
plt.show()



In [ ]:
# Filter data for 2022 and 2023
df_2022 = df[df['Fiscal Year'] == 2022]
df_2023 = df[df['Fiscal Year'] == 2023]

# Calculate weekly sales sum
sales_2022_weekly = df_2022.groupby('Fiscal Week')['Sales Qty'].sum()
sales_2023_weekly = df_2023.groupby('Fiscal Week')['Sales Qty'].sum()

sales_2022_weekly = sales_2022_weekly / sales_2022_weekly.sum()
sales_2023_weekly = sales_2023_weekly / sales_2023_weekly.sum()

# Plot the sales proportion for 2022 and 2023 in subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot sales proportions for 2022
axs[0].bar(sales_2022_weekly.index, sales_2022_weekly, label='2022')
axs[0].set_title('Sales Proportions for 2022')
axs[0].set_xlabel('Fiscal Week')
axs[0].set_ylabel('Sales Proportions')
axs[0].legend()

# Plot sales proportions for 2023
axs[1].bar(sales_2023_weekly.index, sales_2023_weekly, label='2023')
axs[1].set_title('Sales Proportions for 2023')
axs[1].set_xlabel('Fiscal Week')
axs[1].set_ylabel('Sales Proportions')
axs[1].legend()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Concatenate the two Series into a DataFrame
stockout_weekly = pd.concat([stockout_2022_weekly, stockout_2023_weekly], axis=1)
stockout_weekly.columns = ['2022', '2023']

In [ ]:
# Calculate average stockout percent for each group
weekly_stockout_sub = df.groupby(['Fiscal Week', 'Fiscal Year', 'Article: MH6 Sub Product Group'])['Stockout1'].mean() * 100
# Reset index to make 'Fiscal Week' and 'Article: MH6 Sub Product Group' into columns
weekly_stockout_sub = weekly_stockout_sub.reset_index()

# Calculate min, max, and mean of weekly stockout percent for each group
stockout_min_sub = weekly_stockout_sub.groupby('Article: MH6 Sub Product Group')['Stockout1'].min()
stockout_max_sub = weekly_stockout_sub.groupby('Article: MH6 Sub Product Group')['Stockout1'].max()
stockout_mean_sub = weekly_stockout_sub.groupby('Article: MH6 Sub Product Group')['Stockout1'].mean()

# Combine min, max, and mean of weekly stockout percent into one DataFrame
stockout_summary_sub = pd.DataFrame({'Min Stockout%': stockout_min_sub, 'Max Stockout%': stockout_max_sub, 'Mean Stockout%': stockout_mean_sub})

# Display the first 12 rows of the DataFrame
stockout_summary_sub.head(12)

## Sales Pattern

In [ ]:
# Calculate average weekly sales for each group
weekly_sales_sub = df.groupby(['Fiscal Week', 'Fiscal Year', 'Article: MH6 Sub Product Group'])['Sales Qty'].sum()

# Reset index to make 'Fiscal Week' and 'Article: MH6 Sub Product Group' into columns
weekly_sales_sub = weekly_sales_sub.reset_index()

# Calculate min, max, and mean of weekly sales for each group
sales_min_sub = weekly_sales_sub.groupby('Article: MH6 Sub Product Group')['Sales Qty'].min()
sales_max_sub = weekly_sales_sub.groupby('Article: MH6 Sub Product Group')['Sales Qty'].max()
sales_mean_sub = weekly_sales_sub.groupby('Article: MH6 Sub Product Group')['Sales Qty'].mean()

# Combine min, max, and mean of weekly sales into one DataFrame
sales_summary_sub = pd.DataFrame({'Min Sales': sales_min_sub, 'Max Sales': sales_max_sub, 'Mean Sales': sales_mean_sub})

# Display the first 12 rows of the DataFrame
sales_summary_sub.head(12)

## Inventory Pattern

In [ ]:
weekly_inventory_df = df.groupby(['Article', 'Fiscal Year', 'Fiscal Week']).agg({
    'Article: MH5 Product Group': 'first',
    'Article: MH6 Sub Product Group': 'first',
    'Opening Balance Qty': 'first',
    'Closing Balance Qty': 'last',
}).reset_index()

# Calculate min, max, and mean of weekly inventory for each group
inventory_min_sub = weekly_inventory_df.groupby('Article: MH6 Sub Product Group')['Opening Balance Qty'].min()
inventory_max_sub = weekly_inventory_df.groupby('Article: MH6 Sub Product Group')['Opening Balance Qty'].max()
inventory_mean_sub = weekly_inventory_df.groupby('Article: MH6 Sub Product Group')['Opening Balance Qty'].mean()

# Combine min, max, and mean of weekly inventory into one DataFrame
inventory_summary_sub = pd.DataFrame({'Min Inventory': inventory_min_sub, 'Max Inventory': inventory_max_sub, 'Mean Inventory': inventory_mean_sub})

inventory_summary_sub = inventory_summary_sub.round(0)
# Display the first 12 rows of the DataFrame
inventory_summary_sub.head(12)

In [ ]:
inventory_summary_sub.to_excel("Inventory Summary Sub Product Group.xlsx")

In [ ]:
# Export the df to excel
df.to_excel("Outlet X cleaned.xlsx", index=False)

# Censored Demand Estimation Methods

## Intra Week Cyclicity

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming your dataframe is named df
# Set the correct order for weekdays
weekday_order = ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU']
df['Weekday'] = pd.Categorical(df['Weekday'], categories=weekday_order, ordered=True)

# Calculate overall sales weight by weekday
overall_sales_weight = df.groupby('Weekday')['Sales Qty'].sum() / df['Sales Qty'].sum() * 100

# Calculate sales weight by product group
product_group_sales_weight = df.groupby(['Weekday', 'Article: MH5 Product Group'])['Sales Qty'].sum().unstack().apply(lambda x: x / x.sum() * 100)

# Calculate sales weight by sub product group
sub_product_group_sales_weight = df.groupby(['Weekday', 'Article: MH6 Sub Product Group'])['Sales Qty'].sum().unstack().apply(lambda x: x / x.sum() * 100)

# Plot the data
fig, axes = plt.subplots(3, 1, figsize=(10, 12))

# Overall Sales Weight by Weekday
axes[0].bar(overall_sales_weight.index, overall_sales_weight.values)
axes[0].set_title('Overall Sales Weight by Weekday', fontsize=10)
axes[0].set_xlabel('Weekday', fontsize=8)
axes[0].set_ylabel('Sales Weight (%)', fontsize=8)
axes[0].tick_params(axis='both', which='major', labelsize=8)

# Sales Weight by Weekday and Product Group
product_group_sales_weight.plot(kind='bar', ax=axes[1])
axes[1].set_title('Sales Weight by Weekday and Product Group', fontsize=10)
axes[1].set_xlabel('Weekday', fontsize=8)
axes[1].set_ylabel('Sales Weight (%)', fontsize=8)
axes[1].tick_params(axis='both', which='major', labelsize=8)
axes[1].set_xticklabels(product_group_sales_weight.index, rotation=0) 
axes[1].legend(fontsize=6)

# Sales Weight by Weekday and Sub Product Group
sub_product_group_sales_weight.plot(kind='bar', ax=axes[2])
axes[2].set_title('Sales Weight by Weekday and Sub Product Group', fontsize=10)
axes[2].set_xlabel('Weekday', fontsize=8)
axes[2].set_ylabel('Sales Weight (%)', fontsize=8)
axes[2].tick_params(axis='both', which='major', labelsize=8)
axes[2].set_xticklabels(product_group_sales_weight.index, rotation=0) 
axes[2].legend(fontsize=6)

plt.tight_layout()
plt.savefig('sales_weight_by_weekday.png')
plt.show()


In [ ]:
## Calculate the sales weight for each weekday for Fiscal year 2022 and 2023 for all the articles 
# Create a DataFrame for each year
df_2022 = df[df['Fiscal Year'] == 2022]
df_2023 = df[df['Fiscal Year'] == 2023]

# Group by 'Weekday' and calculate the total sales for each weekday
total_sales_weekday_2022 = df_2022.groupby('Weekday')['Sales Qty'].sum()
total_sales_weekday_2023 = df_2023.groupby('Weekday')['Sales Qty'].sum()

# Calculate the total sales for each year
total_sales_2022 = total_sales_weekday_2022.sum()
total_sales_2023 = total_sales_weekday_2023.sum()

# Calculate the sales weight for each weekday
sales_weight_weekday_2022 = (total_sales_weekday_2022 / total_sales_2022*100).round(0)
sales_weight_weekday_2023 = (total_sales_weekday_2023 / total_sales_2023*100).round(0)

sales_weight_weekday_2022, sales_weight_weekday_2023

#Sort the sales weight for each weekday by the day of the week Monday to Sunday "MO" to "SU" and round to 0 decimal places but the sum should be 100

sales_weight_weekday_2022 = sales_weight_weekday_2022.reindex(['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU'])
sales_weight_weekday_2023 = sales_weight_weekday_2023.reindex(['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU'])
sales_weight_weekday_2022, sales_weight_weekday_2023

In [ ]:
# Ensure the sum of sales weight is 100 by rounding down the sales for one weekday
sales_weight_weekday_2022['MO'] = 100 - sales_weight_weekday_2022['TU':'SU'].sum()

sales_weight_weekday_2022

In [ ]:

# Ensure the sum of sales weight is 100 by rounding down the sales for one weekday "TH"
sales_weight_weekday_2023.loc['TH'] = 100 - sales_weight_weekday_2023.loc[['MO', 'TU', 'WE', 'FR', 'SA', 'SU']].sum()

In [ ]:
# Calculate the average sales weight for each weekday
average_sales_weight_weekday = (sales_weight_weekday_2022 + sales_weight_weekday_2023) / 2

# Round the sales weight to 0 decimal places
average_sales_weight_weekday = average_sales_weight_weekday.round(0)

# Ensure the sum of sales weight is 100 by rounding down the sales for one weekday
average_sales_weight_weekday['MO'] = 100 - average_sales_weight_weekday['TU':'SU'].sum()

average_sales_weight_weekday

In [ ]:
# Map the average sales weight to the 'Weekday' column
df['Sales Weight'] = df['Weekday'].map(average_sales_weight_weekday)

In [ ]:
df['Weighted Inventory'] = df['Inv_yn'] * df['Sales Weight']

In [ ]:
df_weekly = df.groupby(['Article', 'Fiscal Year', 'Fiscal Week']).agg({
    'FMS Site': 'first',
    'Calendar Day': 'first',
    'Month': 'first',
    'Month Name': 'first',
    'Fiscal Year Week': 'first',
    'Calendar Year Week': 'first',
    'FMS Site Name': 'first',
    'Gender': 'first',
    'Article: MH5 Product Group': 'first',
    'Article: MH6 Sub Product Group': 'first',
    'Article: Generic ': 'first',
    'Article: Generic Name': 'first',
    'Article: Color Description': 'first',
    'Article Name': 'first',
    'Article: Size ': 'first',
    'Sales Qty': 'sum',
    'Weighted Inventory': 'sum',
    'Opening Balance Qty': 'first',
    'Delta In Transit Qty': 'sum',
    'Closing Balance Qty': 'last',
    'Stockout Days in Week': 'first'
}).reset_index()

#I want to multiply the adjusted sales by 100 to get the percentage of adjusted sales and want to round off the values to 0 decimal places
df_weekly['Adjusted Sales'] = (df_weekly['Sales Qty'] / df_weekly['Weighted Inventory']) * 100
df_weekly['Adjusted Sales'] = df_weekly['Adjusted Sales'].round(0)
df_weekly = df_weekly.sort_values(by=['Article','Fiscal Year', 'Fiscal Week'])

### Lost sales intra-week

In [ ]:
# summation of adjusted sales and sales qty total
adjusted_sales_total = df_weekly['Adjusted Sales'].sum()
sales_qty_total = df_weekly['Sales Qty'].sum()

adjusted_sales_total, sales_qty_total


## Inter- Weekly Impuatation 

In [ ]:
# Group by 'Article: MH5 Product Group', 'Fiscal Year', and 'Fiscal Week' and calculate the total sales for each product group
total_sales_product_group = df.groupby(['Article: MH5 Product Group', 'Fiscal Year', 'Fiscal Week'])['Sales Qty'].sum()
total_sales_product_group = total_sales_product_group.reset_index()
average_sales_product_group = total_sales_product_group.groupby(['Article: MH5 Product Group', 'Fiscal Year'])['Sales Qty'].mean()

In [ ]:
# Merge 'total_sales_product_group' and 'average_sales_product_group' on 'Article: MH5 Product Group' and 'Fiscal Year'
merged_df = pd.merge(total_sales_product_group, average_sales_product_group, on=['Article: MH5 Product Group', 'Fiscal Year'], suffixes=('_total', '_average'))

# Compute the weekly sales rate
merged_df['weekly_sales_rate'] = merged_df['Sales Qty_total'] / merged_df['Sales Qty_average']

# Round the 'weekly_sales_rate' to 1 decimal place
merged_df['weekly_sales_rate'] = merged_df['weekly_sales_rate'].round(1)

# Display the first few rows of the DataFrame
merged_df.head()


In [ ]:
# Group by 'Article: MH5 Product Group', 'Fiscal Year', and 'Fiscal Week' and calculate the total sales for each product group
total_sales_subproduct_group = df.groupby(['Article: MH6 Sub Product Group', 'Fiscal Year', 'Fiscal Week'])['Sales Qty'].sum()

# Reset the index to make the DataFrame easier to work with
total_sales_subproduct_group = total_sales_subproduct_group.reset_index()

# Group by 'Article: MH5 Product Group' and 'Fiscal Year' and calculate the average total sales for each product group
average_sales_subproduct_group = total_sales_subproduct_group.groupby(['Article: MH6 Sub Product Group', 'Fiscal Year'])['Sales Qty'].mean()

# Reset the index to make the DataFrame easier to work with
average_sales_subproduct_group = average_sales_subproduct_group.reset_index()

In [ ]:
# Merge 'total_sales_product_group' and 'average_sales_product_group' on 'Article: MH5 Product Group' and 'Fiscal Year'
merged_df_sub = pd.merge(total_sales_subproduct_group, average_sales_subproduct_group, on=['Article: MH6 Sub Product Group', 'Fiscal Year'], suffixes=('_total', '_average'))

# Compute the weekly sales rate
merged_df_sub['weekly_sales_rate'] = merged_df_sub['Sales Qty_total'] / merged_df_sub['Sales Qty_average']

# Display the first few rows of the DataFrame
merged_df_sub.head()

#round to one decimal place
# Round the 'weekly_sales_rate' to 1 decimal place
merged_df_sub['weekly_sales_rate'] = merged_df_sub['weekly_sales_rate'].round(1)

# Display the first few rows of the DataFrame
merged_df_sub.head()

### Panty Table article vs product group and sub product group

In [ ]:
##Product Group
# Get unique years
years = [2022, 2023]

# Loop over each year
for year in years:
    # Create a new figure for each year
    plt.figure(figsize=(10, 4))

    # Filter the data for the sub product group "CK UDW Men Boxers Boxer Briefs" and the specific year
    product_group_data_year = merged_df[(merged_df['Article: MH5 Product Group'] == 'Women Panty Table') & (merged_df['Fiscal Year'] == year)]

    # Filter the data for the specific article and the specific year
    article_data_year = merged_article_df[merged_article_df['Fiscal Year'] == year]

    # Plot the weekly sales rate for the sub product group
    plt.plot(product_group_data_year['Fiscal Week'], product_group_data_year['weekly_sales_rate'], label=f'Women Panty Table {year}')
    plt.plot(article_data_year['Fiscal Week'], article_data_year['weekly_sales_rate'], label=f'Article X {year}')
    plt.legend()
    plt.title(f'Weekly Sales Rate for  Women Panty Table and Article X for {year}')
    plt.xlabel('Fiscal Week')
    plt.ylabel('Weekly Sales Rate')
    plt.show()

In [ ]:
# Get unique years
years = [2022, 2023]

# Loop over each year
for year in years:
    # Create a new figure for each year
    plt.figure(figsize=(10, 4))

    # Filter the data for the sub product group "CK UDW Men Boxers Boxer Briefs" and the specific year
    sub_product_group_data_year = merged_df_sub[(merged_df_sub['Article: MH6 Sub Product Group'] == 'Women Panty Table Thongs') & (merged_df_sub['Fiscal Year'] == year)]

    # Filter the data for the specific article and the specific year
    article_data_year = merged_article_df[merged_article_df['Fiscal Year'] == year]

    # Plot the weekly sales rate for the sub product group
    plt.plot(sub_product_group_data_year['Fiscal Week'], sub_product_group_data_year['weekly_sales_rate'], label=f'Women Panty Table Thongs {year}')

    # Plot the weekly sales rate for the specific article
    plt.plot(article_data_year['Fiscal Week'], article_data_year['weekly_sales_rate'], label=f'Article X {year}')
    plt.legend()
    plt.title(f'Weekly Sales Rate for Women Panty Table Thongs and Article X for {year}')
    plt.xlabel('Fiscal Week')
    plt.ylabel('Weekly Sales Rate')
    plt.show()

In [ ]:
# Get unique years
years = [2022, 2023]

# Loop over each year
for year in years:
    # Filter the data for the product group "CK UDW Women Panty Table" and the specific year
    product_group_data_year = merged_df[
        (merged_df['Article: MH5 Product Group'] == 'CK UDW Women Panty Table') & 
        (merged_df['Fiscal Year'] == year)
    ]

    # Filter the data for the specific article and the specific year
    article_data_year = merged_article_df[
        merged_article_df['Fiscal Year'] == year
    ]

    # Check if there is data to analyze
    if not product_group_data_year.empty and not article_data_year.empty:
        # Sort data by 'Fiscal Week' to ensure alignment
        product_group_data_year = product_group_data_year.sort_values('Fiscal Week')
        article_data_year = article_data_year.sort_values('Fiscal Week')

        # Merge the dataframes on 'Fiscal Week'
        merged_year_data = pd.merge(
            product_group_data_year, 
            article_data_year, 
            on='Fiscal Week', 
            suffixes=('_prod', '_art')
        )

        # Calculate the correlation between the weekly sales rates of the two datasets
        if not merged_year_data.empty:
            correlation = merged_year_data['weekly_sales_rate_prod'].corr(merged_year_data['weekly_sales_rate_art'])
            print(f"The correlation between weekly sales rates for the product group and the article for {year} is: {correlation:.3f}")
        else:
            print(f"No overlapping data to calculate correlation for the year {year}.")
    else:
        print(f"No data available for either product group or article for {year}.")


In [ ]:
# Filter the DataFrame to include only the rows where 'Stockout Days in Week' is 0
df_no_stockouts = df_weekly[df_weekly['Stockout Days in Week'] == 0]
df_no_stockouts.rename(columns={'Article Name_x': 'Article Name'}, inplace=True)

# Group by 'Article', 'Article Name', 'Fiscal Year', and 'Fiscal Week', then calculate the mean of 'Sales Qty'
average_weekly_sales = df_no_stockouts.groupby(['Article', 'Article Name', 'Fiscal Year'])['Sales Qty'].mean()

# Reset the index to make the DataFrame easier to work with
average_weekly_sales = average_weekly_sales.reset_index()

# Rename the 'Sales Qty' column to 'Average Weekly Sales'
average_weekly_sales.rename(columns={'Sales Qty': 'Average Weekly Sales'}, inplace=True)

In [ ]:
# Merge 'df_weekly' and 'average_weekly_sales' on 'Article' and 'Fiscal Year'
df_weekly = pd.merge(df_weekly, average_weekly_sales, how='left', on=['Article', 'Article Name', 'Fiscal Year'])

In [ ]:
# Merge 'df_weekly' and 'merged_df' on 'Article: MH5 Product Group', 'Fiscal Year', and 'Fiscal Week'
df_weekly = pd.merge(df_weekly, merged_df[['Article: MH5 Product Group', 'Fiscal Year', 'Fiscal Week', 'weekly_sales_rate']], how='left', on=['Article: MH5 Product Group', 'Fiscal Year', 'Fiscal Week'])

In [ ]:
# Calculate the 'Weighted Average Weekly Sales' by multiplying 'Average Weekly Sales' by 'weekly_sales_rate' when 'Stockout Days in Week' is 7
df_weekly['Weighted Average Weekly Sales'] = df_weekly['Average Weekly Sales'] * df_weekly['weekly_sales_rate'] * (df_weekly['Stockout Days in Week'] == 7)
# Round up the values in the 'Weighted Average Weekly Sales' column
df_weekly['Weighted Average Weekly Sales'] = np.ceil(df_weekly['Weighted Average Weekly Sales'])

In [ ]:

# Fill the 'Adjusted Sales' column with the values from the 'Weighted Average Weekly Sales' column where 'Stockout Days in Week' is 7
df_weekly['Adjusted Sales'] = np.where(df_weekly['Stockout Days in Week'] == 7, df_weekly['Weighted Average Weekly Sales'], df_weekly['Adjusted Sales'])
df_weekly.head()

In [ ]:
# summation of adjusted sales and sales qty total for fiscal year 2022 and 2023 
adjusted_sales_total = df_weekly['Adjusted Sales'].sum()
sales_qty_total = df_weekly['Sales Qty'].sum()

adjusted_sales_total, sales_qty_total

In [ ]:
#Lost sales inter/intra weekly
lost_sales = adjusted_sales_total - sales_qty_total
lost_sales

In [ ]:
df_weekly.info()

## Size- Demand Relationship

In [ ]:
df_size = pd.read_excel("Outlet X cleaned.xlsx")

In [ ]:
# Size weight mappings based on Gender and Sizing Type.
size_weights = {
    ('Men', 'sml'): {'L': 0.34, 'M': 0.38, 'S': 0.09, 'XL': 0.17, 'XXL': 0.02},
    ('Women', 'sml'): {'L': 0.19, 'M': 0.37, 'S': 0.34, 'XS': 0.10},
    ('Women', 'brasize'): {
        '32 0A': 0.03, '32 0B': 0.03, '32 0C': 0.04, '32 0D': 0.02, 
        '34 0A': 0.07, '34 0B': 0.13, '34 0C': 0.13, '34 0D': 0.08, 
        '36 0A': 0.04, '36 0B': 0.10, '36 0C': 0.09, '36 0D': 0.07,
        '38 0B': 0.05, '38 0C': 0.07, '38 0D': 0.05
    }
}

def assign_size_weight(row):
    # Extract gender, sizing type, and article size from the row
    gender = row['Gender']
    sizing_type = row['Sizing Type']
    article_size = row['Article: Size ']
    
    # Fetch the weight using the tuple key (Gender, Sizing Type) and the size as the inner key.
    return size_weights.get((gender, sizing_type), {}).get(article_size, None)

# Apply the function across each row to create the new 'Size weight' column
df_size['Size weight'] = df_size.apply(assign_size_weight, axis=1)

In [ ]:
# Now adding the 'Weighted Inventory Size' column
df_size['Weighted Inventory Size'] = df_size['Inv_yn'] * df_size['Size weight']

In [ ]:
# Create a pivot table that sums up the 'Weighted Inventory Size' and 'Sales Qty' for each 'Article: Generic ' and 'Calendar Day'
pivot_table = pd.pivot_table(df_size, values=['Weighted Inventory Size', 'Sales Qty'], index=['Article: Generic ', 'Calendar Day'], aggfunc=np.sum)

# Reset the index of the pivot table to make it easier to merge
pivot_table.reset_index(inplace=True)

# Rename the columns of the pivot table to make them easier to distinguish
pivot_table.rename(columns={'Weighted Inventory Size': 'Weighted Inventory Size_sum', 'Sales Qty': 'Sales Qty_total'}, inplace=True)

# Merge the pivot table with the original DataFrame
df_size = pd.merge(df_size, pivot_table, on=['Article: Generic ', 'Calendar Day'])

# Create the 'Weights' column
df_size['Weights'] = np.where(df_size['Inv_yn'] == 1, 0, df_size['Weighted Inventory Size_sum'])

In [ ]:
# Replace 'inf' values with 0 in the 'Additional Sales' column
df_size['Additional Sales'] = np.ceil((df_size['Sales Qty_total'] / df_size['Weights']) * df_size['Size weight'])
df_size['Additional Sales'] = df_size['Additional Sales'].replace(np.inf, 0)

In [ ]:
# Create the 'Adjusted Sales Size' column
df_size['Adjusted Sales Size'] = df_size['Sales Qty'] + df_size['Additional Sales']

#fill the empty values with 0
df_size['Adjusted Sales Size'] = df_size['Adjusted Sales Size'].fillna(0)
df_size.head()

In [ ]:
#calculate the sum of adjusted sales and sales qty total
adjusted_sales_total = df_size['Adjusted Sales Size'].sum()
sales_qty_total = df_size['Sales Qty'].sum()

adjusted_sales_total, sales_qty_total 


In [ ]:
df_weekly_size = df_size.groupby(['Article', 'Fiscal Year', 'Fiscal Week']).agg({
    'FMS Site': 'first',
    'FMS Site Name': 'first',
    'Gender': 'first',
    'Article: MH5 Product Group': 'first',
    'Article: MH6 Sub Product Group': 'first',
    'Article: Generic ': 'first',
    'Article: Generic Name': 'first',
    'Article: Color Description': 'first',
    'Article Name': 'first',
    'Article: Size ': 'first',
    'Sales Qty': 'sum',
    'Size weight': 'first',
    'Weighted Inventory Size': 'sum',
    'Opening Balance Qty': 'first',
    'Delta In Transit Qty': 'sum',
    'Closing Balance Qty': 'last',
    'Stockout Days in Week': 'first',
    'Adjusted Sales Size': 'sum'
}).reset_index()

df_weekly_size.head()

## Company's current method

In [ ]:
df_us = df.groupby(['Article', 'Fiscal Year', 'Fiscal Week']).agg({
    'Fiscal Year Week': 'first',
    'FMS Site': 'first',
    'FMS Site Name': 'first',
    'Gender': 'first',
    'Article: MH5 Product Group': 'first',
    'Article: MH6 Sub Product Group': 'first',
    'Article: Generic ': 'first',
    'Article: Generic Name': 'first',
    'Article: Color Description': 'first',
    'Article Name': 'first',
    'Article: Size ': 'first',
    'Sales Qty': 'sum',
    'Closing Balance Qty': 'last',
   
}).reset_index()

In [ ]:
df_us.sort_values(by=['Article', 'Fiscal Year', 'Fiscal Week'], inplace=True)

In [ ]:
sr_data = df_us.groupby(['Article', 'Fiscal Year', 'Fiscal Week', 'FMS Site', 'Article: Generic ','Article: Generic Name', 'Article: Color Description', 'Article Name', 'Article: Size '])\
              .agg(Units=('Sales Qty', 'sum'),
                   Inv=('Closing Balance Qty', 'sum')).reset_index()

# Calculate days with positive inventory (denominator for sales rate)
# Assume that negative or 0 inventory at the end of the week implies that inventory ran out the middle of the week
sr_data['Inv.Days'] = sr_data['Inv'].apply(lambda x: 7 if x > 0 else 3.5)

In [ ]:
# Sort the data by 'Fiscal Year', 'Fiscal Week' to ensure the cumulative sum is calculated correctly
sr_data.sort_values(['Article', 'Fiscal Year', 'Fiscal Week'], inplace=True)

# Calculate the cumulative sales by week by year for each article
sr_data['Cumulative Units'] = sr_data.groupby(['Article'])['Units'].cumsum()
sr_data['Cumulative Inv.Days'] = sr_data.groupby(['Article'])['Inv.Days'].cumsum()

In [ ]:
# Calculate the cumulative sales rate as the ceiling of the ratio of cumulative sums multiplied by 7
sr_data['Sales.Rate'] = np.ceil((sr_data['Cumulative Units'] / sr_data['Cumulative Inv.Days']) * 7)
sr_data.fillna(0, inplace=True)

In [ ]:
# Create the 'bop.inv' column
sr_data['bop.inv'] = sr_data.groupby(['Article'])['Inv'].shift(1)

# Fill NaN values with 0
sr_data['bop.inv'].fillna(0, inplace=True)

In [ ]:
# Calculate 'Lost.Sales'
conditions = [
    (sr_data['bop.inv'] <= 0) & (sr_data['Inv'] <= 0),  # BOP & EOP == 0
    ((sr_data['bop.inv'] == 0) | (sr_data['Inv'] == 0)) & ((sr_data['bop.inv'] + sr_data['Inv']) > 0),  # BOP | EOP == 0
    (sr_data['bop.inv'] > 0) & (sr_data['Inv'] > 0)  # BOP & EOP > 0
]
choices = [
    sr_data['Sales.Rate'],  # Lost.Sales = Sales.Rate
    sr_data['Sales.Rate'] / 2,  # Lost.Sales = Sales.Rate / 2
    0  # Lost.Sales = 0
]
sr_data['Lost.Sales'] = np.select(conditions, choices, default=0)

# Round 'Lost.Sales' to 0 decimal places
sr_data['Lost.Sales'] = sr_data['Lost.Sales'].round(0)

In [ ]:
# Create 'Adjusted Sales' column
sr_data['Adjusted Sales'] = sr_data['Lost.Sales'] + sr_data['Units']

In [ ]:
#calculate the sum of adjusted sales and sales qty total
adjusted_sales_total = sr_data['Adjusted Sales'].sum()
sales_qty_total = sr_data['Units'].sum()

# SImulation 

## Simulating stockouts

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot histograms and box plots for stockout days in 2022 and 2023
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(df_weekly['Stockout Days in Week'], kde=True, color='blue', bins=7)
plt.title('2022 Stockout Days Distribution')
plt.xlabel('Stockout Days in Week')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
sns.histplot(df_weekly['Stockout Days in Week'], kde=True, color='green', bins=7)
plt.title('2023 Stockout Days Distribution')
plt.xlabel('Stockout Days in Week')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import multinomial

# Assuming df_weekly_stockouts is already defined
stockout_days = df_weekly_stockouts['Stockout Days in Week'].values

# Filter out zero stockout days
stockout_days = stockout_days[stockout_days > 0]

# Calculate the observed frequencies of each stockout day
unique, counts = np.unique(stockout_days, return_counts=True)

# Total number of observations
n = len(stockout_days)

# Calculate probabilities for the observed stockout days
probabilities = counts / n

# Plotting the observed frequencies and the multinomial PMF
plt.figure(figsize=(10, 6))
sns.histplot(stockout_days, kde=False, bins=range(1, max(stockout_days) + 2), color='g', label='Stockout Data', stat='density')

# Plot the observed probabilities
plt.plot(unique, probabilities, 'ro', ms=8, label='Observed Probabilities')
plt.title('Histogram of Stockout Days in a Week with Observed Probabilities')
plt.xlabel('Number of Stockout Days')
plt.ylabel('Probability')
plt.legend()
plt.show()


## Data prep simulation 

In [ ]:
columns_needed = ['Calendar Day', 'Fiscal Year Week', 'Fiscal Year','Fiscal Week', 'FMS Site','Gender', 'Article: MH5 Product Group', 'Article: MH6 Sub Product Group','Article: Generic ', 'Article: Generic Name', 'Article: Color Description', 'Article', 'Article Name', 'Article: Size ', 'Opening Balance Qty','Sales Qty', 'Closing Balance Qty', 'Inv_yn', 'Stockout1', 'Stockout Days in Week', 'Sales Weight']

# Create a new DataFrame with only these columns
df_needed = df[columns_needed]

# Create a copy of the 'Sales Qty' column for comparison later
df_needed['Original Sales Qty'] = df_needed['Sales Qty']

In [ ]:
#for size to demand relationship accuracy
df_needed1 = df_needed.copy()
df_needed1.head()

In [ ]:
#for inter and intra week cyclicility
# Merge average weekly sales into df_needed
df_needed = pd.merge(df_needed, average_weekly_sales, on=['Article', 'Article Name', 'Fiscal Year'], how='left')

# Merge weekly sales rate into df_needed
df_needed = pd.merge(df_needed, merged_df[['Article: MH5 Product Group', 'Fiscal Year', 'Fiscal Week', 'weekly_sales_rate']], 
                     on=['Article: MH5 Product Group', 'Fiscal Year', 'Fiscal Week'], how='left')

In [ ]:
#drop the sales weight column from df_needed1
df_needed1.drop(columns='Sales Weight', inplace=True)

In [ ]:
df_needed1['Sizing Type']= df_needed1['Article: MH6 Sub Product Group'].map({
'Men Boxers Briefs': 'sml',
' Men Hip Briefs': 'sml',
'Men Low Rise Trunks': 'sml',
'Men High Rise Trunks': 'sml',
'Women Coordinate Boy Shorts': 'sml',
'Women Coordinate Briefs': 'sml',
'Women Coordinate Thongs':'sml',
'Women Panty Table Briefs': 'sml',
'Women Panty Table Hipsters': 'sml',
'Women Panty Table Thongs':'sml',
'Women Bralette Bras': 'sml',
'Women Other Bras': 'brasize',
})

df_needed1.head()

In [ ]:
size_weights = {
    ('Men', 'sml'): {'L': 0.34, 'M': 0.38, 'S': 0.09, 'XL': 0.17, 'XXL': 0.02},
    ('Women', 'sml'): {'L': 0.19, 'M': 0.37, 'S': 0.34, 'XS': 0.10},
    ('Women', 'brasize'): {
        '32 0A': 0.03, '32 0B': 0.03, '32 0C': 0.04, '32 0D': 0.02, 
        '34 0A': 0.07, '34 0B': 0.13, '34 0C': 0.13, '34 0D': 0.08, 
        '36 0A': 0.04, '36 0B': 0.10, '36 0C': 0.09, '36 0D': 0.07,
        '38 0B': 0.05, '38 0C': 0.07, '38 0D': 0.05
    }
}
def assign_size_weight(row):
    # Extract gender, sizing type, and article size from the row
    gender = row['Gender']
    sizing_type = row['Sizing Type']
    article_size = row['Article: Size ']
    
    # Fetch the weight using the tuple key (Gender, Sizing Type) and the size as the inner key.
    return size_weights.get((gender, sizing_type), {}).get(article_size, None)

# Apply the function across each row to create the new 'Size weight' column
df_needed1['Size weight'] = df_needed1.apply(assign_size_weight, axis=1)
df_needed1.head()

## Intra-Inter Week Simulation

In [ ]:
import numpy as np
import pandas as pd
import rando

def run_simulation(df_needed, iterations, probabilities, seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    results = []
    wmape_list = []
    wmpe_list = []
    df_copy_list = []

    for _ in range(iterations):
        df_copy = df_needed.copy()
        df_copy['Simulated Stockout Days in Week'] = 0

        # Filter out weeks where the articles were in stock for the entire week
        df_copy = df_copy[df_copy['Stockout Days in Week'] == 0]

        # For each article, year, and week, simulate stockout days using Multinomial distribution
        weekly_groups = df_copy.groupby(['Article', 'Fiscal Year', 'Fiscal Week'])
        for (article, year, week), group in weekly_groups:
            # Draw number of stockout days using multinomial distribution
            num_days_to_stockout = np.argmax(np.random.multinomial(1, probabilities)) + 1
            daily_indices = list(group.index)
            if len(daily_indices) >= num_days_to_stockout:
                simulated_stockout_indices = random.sample(daily_indices, num_days_to_stockout)
                df_copy.loc[simulated_stockout_indices, 'Inv_yn'] = 0
                df_copy.loc[simulated_stockout_indices, 'Sales Qty'] = 0
                df_copy.loc[simulated_stockout_indices, 'Simulated Stockout Days in Week'] += 1

        # Calculate weighted inventory for intra-week cyclicity
        df_copy['Weighted Inventory'] = df_copy['Inv_yn'] * df_copy['Sales Weight']

        # Aggregate the data to weekly-level DataFrame
        df_weekly1 = df_copy.groupby(['Article', 'Fiscal Year', 'Fiscal Week']).agg({
            'FMS Site': 'first',
            'Sales Qty': 'sum',
            'Article: MH5 Product Group': 'first',
            'Article: MH6 Sub Product Group': 'first',
            'Article: Generic ': 'first',
            'Article: Generic Name': 'first',
            'Article: Color Description': 'first',
            'Original Sales Qty': 'sum',
            'Weighted Inventory': 'sum',
            'Inv_yn': 'last',
            'Stockout Days in Week': 'first',
            'Simulated Stockout Days in Week': 'sum',
            'Average Weekly Sales': 'first',
            'weekly_sales_rate': 'first'
        }).reset_index()
        
        # Apply adjustments for intra-week and inter-week cyclicity
        df_weekly1['Adjusted Sales'] = df_weekly1.apply(
            lambda row: (row['Average Weekly Sales'] * row['weekly_sales_rate']
                         if row['Simulated Stockout Days in Week'] == 7
                         else (row['Sales Qty'] / row['Weighted Inventory']) * 100),
            axis=1
        )
        df_weekly1['Adjusted Sales'] = df_weekly1['Adjusted Sales'].round(0)
        df_weekly1['Difference'] = df_weekly1['Adjusted Sales'] - df_weekly1['Original Sales Qty']

        # Calculate error metrics
        mae = np.mean(np.abs(df_weekly1['Difference']))
        rmse = np.sqrt(np.mean(df_weekly1['Difference']**2))
        mape = np.mean(np.abs(df_weekly1['Difference'] / df_weekly1['Original Sales Qty'])) * 100
        wmape = np.sum(np.abs(df_weekly1['Difference'])) / np.sum(df_weekly1['Original Sales Qty']) * 100
        wmpe = np.sum(df_weekly1['Difference']) / np.sum(df_weekly1['Original Sales Qty']) * 100

        results.append(df_weekly1['Difference'].mean())
        wmape_list.append(wmape)
        wmpe_list.append(wmpe)
        df_copy_list.append(df_copy)

    return results, mae_list, rmse_list, mape_list, wmape_list, wmpe_list, df_weekly1, df_copy_list

# Probabilities for the multinomial distribution
probabilities = [0.16, 0.10, 0.08, 0.06, 0.10, 0.02, 0.48]
seed_value = 12345

results, wmape_list, wmpe_list, df_weekly1, df_copy_list = run_simulation(df_needed, 100, probabilities, seed_value)

# Display results
average_difference = np.mean(results)
std_dev_difference = np.std(results)
average_wmape = np.mean(wmape_list)
average_wmpe = np.mean(wmpe_list)

print(f'Average Difference: {average_difference}')
print(f'Standard Deviation of Difference: {std_dev_difference}')
print(f'Average wMAPE: {average_wmape}')
print(f'Average wMPE: {average_wmpe}')

# Access df_copy_list for the last iteration's daily-level data
final_df_copy = df_copy_list[-1]
final_df_copy.head()


## Company methodology

In [ ]:
import numpy as np
import pandas as pd
import random

def run_simulation(df_needed, iterations, probabilities, seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    results = []
    wmape_list = []
    wmpe_list = []
    df_copy_list = []

    for _ in range(iterations):
        df_copy = df_needed.copy()
        df_copy['Simulated Stockout Days in Week'] = 0

        # Filter out articles that were in stock for the entire week
        df_copy = df_copy[df_copy['Stockout Days in Week'] == 0]

        # For each article, year, and week, simulate stockout days using Multinomial distribution
        weekly_groups = df_copy.groupby(['Article', 'Fiscal Year', 'Fiscal Week'])
        for (article, year, week), group in weekly_groups:
            # Draw number of stockout days using multinomial distribution
            num_days_to_stockout = np.argmax(np.random.multinomial(1, probabilities)) + 1
            daily_indices = list(group.index)
            if len(daily_indices) >= num_days_to_stockout:
                simulated_stockout_indices = random.sample(daily_indices, num_days_to_stockout)
                df_copy.loc[simulated_stockout_indices, 'Inv_yn'] = 0
                df_copy.loc[simulated_stockout_indices, 'Sales Qty'] = 0
                df_copy.loc[simulated_stockout_indices, 'Simulated Stockout Days in Week'] += 1

        # Calculate weighted inventory for intra-week cyclicity
        df_copy['Weighted Inventory'] = df_copy['Inv_yn'] * df_copy['Sales Weight']

        # Aggregate the data to weekly-level DataFrame
        df_weekly1 = df_copy.groupby(['Article', 'Fiscal Year', 'Fiscal Week']).agg({
            'FMS Site': 'first',
            'Sales Qty': 'sum',
            'Article: MH5 Product Group': 'first',
            'Article: MH6 Sub Product Group': 'first',
            'Article: Generic ': 'first',
            'Article: Generic Name': 'first',
            'Article: Color Description': 'first',
            'Original Sales Qty': 'sum',
            'Weighted Inventory': 'sum',
            'Inv_yn': 'last',
            'Stockout Days in Week': 'first',
            'Simulated Stockout Days in Week': 'sum',
            'Average Weekly Sales': 'first',
            'weekly_sales_rate': 'first'
        }).reset_index()
        
        # Apply adjustments for intra-week and inter-week cyclicity
        df_weekly1.rename(columns={'Inv_yn': 'Inventory Closing'}, inplace=True)
        df_weekly1['Inv.days'] = np.where(df_weekly1['Inventory Closing'] == 0, 3.5, 7)
        df_weekly1.sort_values(['Article', 'Fiscal Year', 'Fiscal Week'], inplace=True)

        # Calculate the cumulative sales by week by year for each article
        df_weekly1['Cumulative Sales'] = df_weekly1.groupby('Article')['Sales Qty'].cumsum()
        df_weekly1['Cumulative Inv.days'] = df_weekly1.groupby('Article')['Inv.days'].cumsum()
        
        # Calculate the cumulative sales rate as the ceiling of the ratio of cumulative sums multiplied by 7
        df_weekly1['Sales Rate'] = np.ceil(df_weekly1['Cumulative Sales'] / df_weekly1['Cumulative Inv.days'] * 7)
        df_weekly1['Sales Rate'].fillna(0, inplace=True)
        
        # Create the 'bop.inv' column and fill the 
        df_weekly1['bop.inv'] = df_weekly1.groupby('Article')['Inventory Closing'].shift(1)
        df_weekly1['bop.inv'].fillna(0, inplace=True)
        
        #calculate the lost sales conditions 
        conditions= [
            (df_weekly1['bop.inv'] == 0) & (df_weekly1['Inventory Closing'] == 0), #BOP and closing inventory =0 
            # BOP or closing inventory =0 either one of them is zero
            (df_weekly1['bop.inv'] == 0) | (df_weekly1['Inventory Closing'] == 0),
            #BOP and colosing =1
            (df_weekly1['bop.inv'] == 1) & (df_weekly1['Inventory Closing'] == 1), 
            ]                   
        # in first condition the lost sales is the sales rate
        #in secoond condition lost sales is sales rate/2
        #in third condition lost sales is 0
        choices =[
            df_weekly1['Sales Rate'],
            df_weekly1['Sales Rate']/2,
            0
        ]

        df_weekly1['Lost Sales'] = np.select(conditions, choices, default=0)

        #round the lost sales to 0 decimal places
        df_weekly1['Lost Sales'] = df_weekly1['Lost Sales'].round(0)

        df_weekly1.head()

        df_weekly1['Adjusted Sales'] = df_weekly1['Sales Qty'] + df_weekly1['Lost Sales']
        df_weekly1['Adjusted Sales'] = df_weekly1['Adjusted Sales'].round(0)
        
        df_weekly1['Difference'] = df_weekly1['Adjusted Sales'] - df_weekly1['Original Sales Qty']
        # Calculate error metrics
        wmape = np.sum(np.abs(df_weekly1['Difference'])) / np.sum(df_weekly1['Original Sales Qty']) * 100
        wmpe = np.sum(df_weekly1['Difference']) / np.sum(df_weekly1['Original Sales Qty']) * 100

        results.append(df_weekly1['Difference'].mean())
        wmape_list.append(wmape)
        wmpe_list.append(wmpe)
        df_copy_list.append(df_copy)

    return results, wmape_list, wmpe_list, df_weekly1, df_copy_list

# Set the probabilities for the multinomial distribution
probabilities = [0.16, 0.10, 0.08, 0.06, 0.10, 0.02, 0.48]
seed_value = 12345

results, wmape_list, wmpe_list, df_weekly1, df_copy_list = run_simulation(df_needed, 100, probabilities, seed_value)

# Display results
average_difference = np.mean(results)
std_dev_difference = np.std(results)
average_wmape = np.mean(wmape_list)
average_wmpe = np.mean(wmpe_list)

print(f'Average Difference: {average_difference}')
print(f'Standard Deviation of Difference: {std_dev_difference}')
print(f'Average wMAPE: {average_wmape}')
print(f'Average wMPE: {average_wmpe}')

# Access df_copy_list for the last iteration's daily-level data
final_df_copy = df_copy_list[-1]
final_df_copy.head()


## Size Demand Simulation

In [ ]:
import numpy as np
import pandas as pd
import random

def run_simulation(df_needed1, iterations, probabilities, seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    results = []
    wmape_list = []
    wmpe_list = []
    df_size_list = []

    for _ in range(iterations):
        df_size1 = df_needed1.copy()
        df_size1['Simulated Stockout Days in Week'] = 0

        # Filter out articles that were in stock for the entire week
        df_size1 = df_size1[df_size1['Stockout Days in Week'] == 0]

        # For each article, year, and week, simulate stockout days using Multinomial distribution
        weekly_groups = df_size1.groupby(['Article', 'Fiscal Year', 'Fiscal Week'])
        for (article, year, week), group in weekly_groups:
            # Draw number of stockout days using multinomial distribution
            num_days_to_stockout = np.argmax(np.random.multinomial(1, probabilities)) + 1
            daily_indices = list(group.index)
            if len(daily_indices) >= num_days_to_stockout:
                simulated_stockout_indices = random.sample(daily_indices, num_days_to_stockout)
                df_size1.loc[simulated_stockout_indices, 'Inv_yn'] = 0
                df_size1.loc[simulated_stockout_indices, 'Sales Qty'] = 0
                df_size1.loc[simulated_stockout_indices, 'Simulated Stockout Days in Week'] += 1

        # Calculate weighted inventory for intra-week cyclicity
        df_size1['Weighted Inventory Size'] = df_size1['Inv_yn'] * df_size1['Size weight']
        
        grouped = df_size1.groupby(['Article: Generic ', 'Calendar Day']).agg({
        'Weighted Inventory Size': 'sum',
        'Sales Qty': 'sum'
                    }).reset_index()
        
        grouped.rename(columns={
            'Weighted Inventory Size': 'Weighted Inventory Size_sum',
            'Sales Qty': 'Sales Qty_total'
                }, inplace=True)

        df_size1 = pd.merge(df_size1, grouped, on=['Article: Generic ', 'Calendar Day'], how='left')
        
        df_size1['Weights'] = np.where(df_size1['Inv_yn'] == 1, 0, df_size1['Weighted Inventory Size_sum'])

        df_size1['Additional Sales'] = np.ceil((df_size1['Sales Qty_total'] / df_size1['Weights']) * df_size1['Size weight'])
        df_size1['Additional Sales'] = df_size1['Additional Sales'].replace(np.inf, 0)
        df_size1['Adjusted Sales Size'] = df_size1['Sales Qty'] + df_size1['Additional Sales']


        # Create the weekly-level DataFrame by aggregating data
        df_weekly_size1 = df_size1.groupby(['Article', 'Fiscal Year', 'Fiscal Week']).agg({
            'FMS Site': 'first',
            'Article: MH5 Product Group': 'first',
            'Article: MH6 Sub Product Group': 'first',
            'Article: Generic ': 'first',
            'Article: Generic Name': 'first',
            'Article: Color Description': 'first',
            'Article Name': 'first',
            'Article: Size ': 'first',
            'Sales Qty': 'sum',
            'Original Sales Qty': 'sum',
            'Size weight': 'first',
            'Weighted Inventory Size': 'sum',
            'Stockout Days in Week': 'first',
            'Simulated Stockout Days in Week': 'sum',
            'Adjusted Sales Size': 'sum'
        
        }).reset_index()
        
        df_weekly_size1['Difference'] = df_weekly_size1['Adjusted Sales Size'] - df_weekly_size1['Original Sales Qty']

        # Calculate error metrics
        mae = np.mean(np.abs(df_weekly_size1['Difference']))
        rmse = np.sqrt(np.mean(df_weekly_size1['Difference']**2))
        mape = np.mean(np.abs(df_weekly_size1['Difference'] / df_weekly_size1['Original Sales Qty'])) * 100
        wmape = np.sum(np.abs(df_weekly_size1['Difference'])) / np.sum(df_weekly_size1['Original Sales Qty']) * 100
        wmpe = np.sum(df_weekly_size1['Difference']) / np.sum(df_weekly_size1['Original Sales Qty']) * 100
        
        results.append(df_weekly_size1['Difference'].mean())
        mae_list.append(mae)
        rmse_list.append(rmse)
        mape_list.append(mape)
        wmape_list.append(wmape)
        wmpe_list.append(wmpe)
        df_size_list.append(df_size1)

    return results, mae_list, rmse_list, mape_list, wmape_list, wmpe_list, df_weekly_size1, df_size_list

# Set the probabilities for the multinomial distribution
probabilities = [0.16, 0.10, 0.08, 0.06, 0.10, 0.02, 0.48]
seed_value = 12345

results, wmape_list, wmpe_list, df_weekly_size1, df_size_list = run_simulation(df_needed1, 10, probabilities, seed_value)

# Display results
average_difference = np.mean(results)
std_dev_difference = np.std(results)
average_wmape = np.mean(wmape_list)
average_wmpe = np.mean(wmpe_list)

print(f'Average Difference: {average_difference}')
print(f'Standard Deviation of Difference: {std_dev_difference}')
print(f'Average wMAPE: {average_wmape}')
print(f'Average wMPE: {average_wmpe}')

# Access df_copy_list for the last iteration's daily-level data
final_size_copy = df_size_list[-1]
final_size_copy.head()


## Simulation Approach 2 Simulating demand 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import Workbook

def simulate_demand(sku_sales, weekly_proportions, weekday_proportions):
    # Ensure the proportions sum to 1
    if not (np.isclose(sum(weekly_proportions), 1) and np.isclose(sum(weekday_proportions), 1)):
        raise ValueError("Proportions must sum to 1.")
    
    # Calculate total weeks in a year
    total_weeks = 52
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    
    # Initialize a list to store results
    demand_simulation = []

    # Iterate over each SKU
    for index, row in sku_sales.iterrows():
        sku = row['SKU']
        total_annual_sales = row['Annual_Sales']
        
        # Generate weekly demand based on multinomial distribution
        weekly_demand = np.random.multinomial(total_annual_sales, weekly_proportions)
        
        # Simulate daily demand within each week using multinomial distribution
        for week in range(total_weeks):
            daily_demand = np.random.multinomial(weekly_demand[week], weekday_proportions)
            for day in range(7):
                demand_simulation.append({'SKU': sku, 'Week': week+1, 'Day': weekdays[day], 'Demand': daily_demand[day]})
    
    # Convert list to DataFrame
    demand_simulation_df = pd.DataFrame(demand_simulation)
    
    return demand_simulation_df

# Example SKU sales data
sku_sales = pd.DataFrame({
    'SKU': [
        '01', '02', '03', '04',
        '05', '06', '07', '08',
        '08', '09', '10', '11', '12'
    ],
    'Annual_Sales': [200, 180, 90, 50, 260, 260, 80, 170, 220, 190, 50, 180]
})

# Example: Custom weekly and weekday proportions
weekly_proportions = [1/52] * 52 

weekday_proportions = [0.12, 0.10, 0.10, 0.10, 0.13, 0.19, 0.26] 

# Simulate demand
np.random.seed(123) # Set seed for reproducibility
demand_simulation_df = simulate_demand(sku_sales, weekly_proportions, weekday_proportions)

# View the first few rows of the simulation result
print(demand_simulation_df.head())

# Check the total demand to ensure it matches the total annual sales
total_demand_check = demand_simulation_df.groupby('SKU')['Demand'].sum().reset_index()
print(total_demand_check)

# Aggregate demand by weekday for plotting
weekday_demand = demand_simulation_df.groupby('Day')['Demand'].sum().reset_index()
weekday_demand['Proportion'] = weekday_demand['Demand'] / weekday_demand['Demand'].sum()
weekday_demand['Day'] = pd.Categorical(weekday_demand['Day'], categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], ordered=True)
weekday_demand = weekday_demand.sort_values('Day')

# Plot the proportion of sales per weekday
plt.figure(figsize=(10, 6))
plt.bar(weekday_demand['Day'], weekday_demand['Proportion'], color='skyblue')
for index, value in enumerate(weekday_demand['Proportion']):
    plt.text(index, value, f'{value:.2%}', ha='center', va='bottom')
plt.title('Proportion of Sales per Weekday')
plt.xlabel('Weekday')
plt.ylabel('Proportion of Total Sales')
plt.ylim(0, max(weekday_demand['Proportion']) + 0.05)
plt.show()

demand_simulation_df.to_excel("Sim_demand(12SKUS).xlsx", index=False)


In [ ]:
df = pd.read_excel("simulation_final.xlsx")


In [ ]:
# Group by 'Weekday' and calculate the total sales for each weekday
total_sales_weekday = df.groupby('Weekday')['Sales'].sum()

# Calculate the total sales for each year
total_sales = total_sales_weekday.sum()

# Calculate the sales weight for each weekday
sales_weight_weekday = (total_sales_weekday / total_sales*100).round(0)

In [ ]:
# Map the average sales weight to the 'Weekday' column
df['Sales Weight'] = df['Weekday'].map(sales_weight_weekday)

#reset te index of df since it is giving weird index values
df.reset_index(drop=True, inplace=True)
# make a new column inv_yn which is 1 if opening inventory is greater than 0 and 0 otherwise
df['Inv_yn'] = df['Opening Inventory'].apply(lambda x: 1 if x > 0 else 0)
df['Weighted Inventory'] = df['Inv_yn'] * df['Sales Weight']

In [ ]:
df_weekly = df.groupby(['Article', 'Fiscal Week']).agg({
    'Article: MH5 Product Group': 'first',
    'Article: Generic ': 'first',
    'Sales': 'sum',
    'Demand': 'sum',
    'Weighted Inventory': 'sum',
    'Opening Inventory': 'first',
    'Replenishement Qty': 'sum',
    'Closing Inventory': 'last',
    'Stockout': 'sum',
}).reset_index()

#I want to multiply the adjusted sales by 100 to get the percentage of adjusted sales and want to round off the values to 0 decimal places
df_weekly['Adjusted Sales'] = (df_weekly['Sales'] / df_weekly['Weighted Inventory']) * 100
df_weekly['Adjusted Sales'] = df_weekly['Adjusted Sales'].round(0)
df_weekly = df_weekly.sort_values(by=['Article', 'Fiscal Week'])

In [ ]:
# Group by 'Article: MH5 Product Group', 'Fiscal Year', and 'Fiscal Week' and calculate the total sales for each product group
total_sales_product_group = df.groupby(['Article: MH5 Product Group', 'Fiscal Week'])['Sales'].sum()

# Reset the index to make the DataFrame easier to work with
total_sales_product_group = total_sales_product_group.reset_index()

# Display the first few rows of the DataFrame
total_sales_product_group.head()

average_sales_product_group = total_sales_product_group.groupby(['Article: MH5 Product Group'])['Sales'].mean()
merged_df = pd.merge(total_sales_product_group, average_sales_product_group, on=['Article: MH5 Product Group'], suffixes=('_total', '_average'))

# Compute the weekly sales rate
merged_df['weekly_sales_rate'] = merged_df['Sales_total'] / merged_df['Sales_average']

# Round the 'weekly_sales_rate' to 1 decimal place
merged_df['weekly_sales_rate'] = merged_df['weekly_sales_rate'].round(1)


In [ ]:
df_no_stockouts = df_weekly[df_weekly['Stockout'] == 0]

In [ ]:
# Group by 'Article', 'Article Name', 'Fiscal Year', and 'Fiscal Week', then calculate the mean of 'Sales Qty'
average_weekly_sales = df_no_stockouts.groupby(['Article'])['Sales'].mean()

# Reset the index to make the DataFrame easier to work with
average_weekly_sales = average_weekly_sales.reset_index()

# Rename the 'Sales Qty' column to 'Average Weekly Sales'
average_weekly_sales.rename(columns={'Sales': 'Average Weekly Sales'}, inplace=True)

In [ ]:
df_weekly = pd.merge(df_weekly, average_weekly_sales, how='left', on=['Article'])

In [ ]:
df_weekly = pd.merge(df_weekly, merged_df[['Article: MH5 Product Group', 'Fiscal Week', 'weekly_sales_rate']], how='left', on=['Article: MH5 Product Group', 'Fiscal Week'])

In [ ]:
df_weekly['Weighted Average Weekly Sales'] = df_weekly['Average Weekly Sales'] * df_weekly['weekly_sales_rate'] * (df_weekly['Stockout'] == 7)

# Round up the values in the 'Weighted Average Weekly Sales' column
df_weekly['Weighted Average Weekly Sales'] = np.ceil(df_weekly['Weighted Average Weekly Sales'])
df_weekly.head()

In [ ]:
df_weekly['Adjusted Sales'] = np.where(df_weekly['Stockout'] == 7, df_weekly['Weighted Average Weekly Sales'], df_weekly['Adjusted Sales'])
df_weekly.head()

In [ ]:
size_weights = {
    'S': 0.1,
    'M': 0.41,
    'L': 0.34,
    'XL': 0.15
}
df['Size weight'] = df['Article: Size '].map(size_weights)
df_size['Weighted Inventory Size'] = df_size['Inv_yn'] * df_size['Size weight']

In [ ]:
# Create a pivot table that sums up the 'Weighted Inventory Size' and 'Sales Qty' for each 'Article: Generic ' and 'Calendar Day'
pivot_table = pd.pivot_table(df_size, values=['Weighted Inventory Size', 'Sales'], index=['Article: Generic ', 'Calendar Day'], aggfunc=np.sum)

# Reset the index of the pivot table to make it easier to merge
pivot_table.reset_index(inplace=True)

# Rename the columns of the pivot table to make them easier to distinguish
pivot_table.rename(columns={'Weighted Inventory Size': 'Weighted Inventory Size_sum', 'Sales': 'Sales_total'}, inplace=True)

# Merge the pivot table with the original DataFrame
df_size = pd.merge(df_size, pivot_table, on=['Article: Generic ', 'Calendar Day'])

# Create the 'Weights' column
df_size['Weights'] = np.where(df_size['Inv_yn'] == 1, 0, df_size['Weighted Inventory Size_sum'])

df_size.head()

In [ ]:
# Replace 'inf' values with 0 in the 'Additional Sales' column
df_size['Additional Sales'] = np.ceil((df_size['Sales_total'] / df_size['Weights']) * df_size['Size weight'])

In [ ]:
# Create the 'Adjusted Sales Size' column
df_size['Adjusted Sales Size'] = df_size['Sales'] + df_size['Additional Sales']

#fill the empty values with 0
df_size['Adjusted Sales Size'] = df_size['Adjusted Sales Size'].fillna(0)

In [ ]:
df_size_weekly= df_size.groupby(['Article', 'Fiscal Week']).agg({
    'Article: MH5 Product Group': 'first',
    'Article: Generic ': 'first',
    'Sales': 'sum',
    'Demand': 'sum',
    'Adjusted Sales Size': 'sum',
    'Opening Inventory': 'first',
    'Replenishement Qty': 'sum',
    'Closing Inventory': 'last',
    'Stockout': 'sum',
}).reset_index()

In [ ]:
df_us = df.groupby(['Article', 'Fiscal Week']).agg({
    'Article: MH5 Product Group': 'first',
    'Article: Generic ': 'first',
    'Article: Size ': 'first',
    'Sales': 'sum',
    'Demand': 'sum',
    'Stockout': 'sum',
    'Closing Inventory': 'last',
   
}).reset_index()

In [ ]:
# Calc units and inventory by type, fiscal week, site, vendor style
sr_data = df_us.groupby(['Article', 'Fiscal Week', 'Article: Generic ', 'Article: Size ','Demand'])\
              .agg(Units=('Sales', 'sum'),
                   Inv=('Closing Inventory', 'sum'),
                        Stockout=('Stockout','sum')).reset_index()

# Calculate days with positive inventory (denominator for sales rate)
# Assume that negative or 0 inventory at the end of the week implies that inventory ran out the middle of the week
sr_data['Inv.Days'] = sr_data['Inv'].apply(lambda x: 7 if x > 0 else 3.5)

sr_data.head()

In [ ]:
# Sort the data by 'Fiscal Year', 'Fiscal Week' to ensure the cumulative sum is calculated correctly
sr_data.sort_values(['Article', 'Fiscal Week'], inplace=True)

# Calculate the cumulative sales by week by year for each article
sr_data['Cumulative Units'] = sr_data.groupby(['Article'])['Units'].cumsum()
sr_data['Cumulative Inv.Days'] = sr_data.groupby(['Article'])['Inv.Days'].cumsum()

In [ ]:
# Calculate the cumulative sales rate as the ceiling of the ratio of cumulative sums multiplied by 7
sr_data['Sales.Rate'] = np.ceil((sr_data['Cumulative Units'] / sr_data['Cumulative Inv.Days']) * 7)
sr_data.fillna(0, inplace=True)

In [ ]:
# Create the 'bop.inv' column
sr_data['bop.inv'] = sr_data.groupby(['Article'])['Inv'].shift(1)

# Fill NaN values with 0
sr_data['bop.inv'].fillna(0, inplace=True)

In [ ]:
# Calculate 'Lost.Sales'
conditions = [
    (sr_data['bop.inv'] <= 0) & (sr_data['Inv'] <= 0),  # BOP & EOP == 0
    ((sr_data['bop.inv'] == 0) | (sr_data['Inv'] == 0)) & ((sr_data['bop.inv'] + sr_data['Inv']) > 0),  # BOP | EOP == 0
    (sr_data['bop.inv'] > 0) & (sr_data['Inv'] > 0)  # BOP & EOP > 0
]
choices = [
    sr_data['Sales.Rate'],  # Lost.Sales = Sales.Rate
    sr_data['Sales.Rate'] / 2,  # Lost.Sales = Sales.Rate / 2
    0  # Lost.Sales = 0
]
sr_data['Lost.Sales'] = np.select(conditions, choices, default=0)

# Round 'Lost.Sales' to 0 decimal places
sr_data['Lost.Sales'] = sr_data['Lost.Sales'].round(0)

In [ ]:
# Create 'Adjusted Sales' column
sr_data['Adjusted Sales'] = sr_data['Lost.Sales'] + sr_data['Units']

In [ ]:
# Calculate the difference between Adjusted sales and demand 
sr_data['Difference'] =  sr_data['Demand'] - sr_data['Adjusted Sales']

wmape = np.sum(np.abs(sr_data['Difference'])) / np.sum(sr_data['Demand']) * 100
wmpe = np.sum(sr_data['Difference']) / np.sum(sr_data['Demand']) * 100

wmape, wmpe

In [ ]:
# Calculate the difference between Adjusted sales and demand 
df_weekly['Difference'] =  df_weekly['Demand'] - df_weekly['Adjusted Sales']

wmape = np.sum(np.abs(df_weekly['Difference'])) / np.sum(df_weekly['Demand']) * 100
wmpe = np.sum(df_weekly['Difference']) / np.sum(df_weekly['Demand']) * 100

wmape, wmpe

In [ ]:
# Calculate the difference between Adjusted sales and demand 
df_size_weekly['Difference'] =  df_size_weekly['Demand'] - df_size_weekly['Adjusted Sales Size']

wmape = np.sum(np.abs(df_size_weekly['Difference'])) / np.sum(df_size_weekly['Demand']) * 100
wmpe = np.sum(df_size_weekly['Difference']) / np.sum(df_size_weekly['Demand']) * 100

wmape, wmpe